# Дёргальщик RusVectores-эмбеддингов
### **2025-01-22**
См::<br>
RusVectōrēs: семантические модели для русского языка<br>
Елизавета Кузьменко, Андрей Кутузов
<br>
https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb


In [1]:
import sys, gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#import zipfile
#model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
#m = wget.download(model_url)
#model_file = model_url.split('/')[-1]
#model_file = "180.zip"

"""

NB


Из zip-архива у меня не заработало.
Тогда из файла 'http://vectors.nlpl.eu/repository/11/180.zip' 
вручную выложил в директорию '180_model.bin'


"""

extracted_model_file = '180_model.bin'

#with zipfile.ZipFile(model_file, 'r') as archive:
#Extract the model.bin file to the current directory
#archive.extract('model.bin', path='.')

# Now load the extracted model file
model = gensim.models.KeyedVectors.load_word2vec_format(extracted_model_file, binary=True)

2025-01-23 23:43:39,043 : INFO : loading projection weights from 180_model.bin
2025-01-23 23:43:42,870 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (189193, 300) matrix of type float32 from 180_model.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2025-01-23T23:43:42.870286', 'gensim': '4.3.3', 'python': '3.8.20 | packaged by conda-forge | (default, Sep 30 2024, 17:44:03) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'load_word2vec_format'}


In [2]:
#pip install pymystem3 requests
import numpy as np
import requests
from pymystem3 import Mystem

def tag_mystem(text="Текст нужно передать функции в виде строки!", mapping=None, postags=True):
    processed = m.analyze(text)
    #https://github.com/nlpub/pymystem3/blob/master/pymystem3/mystem.py
    tagged = []
    # если частеречные тэги не нужны (например, их нет в модели), выставьте postags=False
    # в этом случае на выход будут поданы только леммы
   
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(",")[0]
            pos = pos.split("=")[0].strip()
            
            if mapping:
                if pos in mapping:
                    pos = mapping[pos]
                else:
                    pos = "X"
                    
            tagged.append(lemma.lower() + "_" + pos)
        except KeyError:
            continue
            
    if not postags:
        tagged = [t.split("_")[0] for t in tagged]
    
    return tagged

# Таблица преобразования частеречных тэгов Mystem в тэги UPoS:
#mapping_url = "https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map"
#mystem2upos = {}
#r = requests.get(mapping_url, stream=True)

#for pair in r.text.split("\n"):
#    pair = pair.split()
#    if len(pair) > 1:
#        mystem2upos[pair[0]] = pair[1]
# Захардкодим таблицу преобразования частеречных тегов Mystem в теги UPoS
mystem2upos = {
    "A": "ADJ",
    "ADV": "ADV",
    "ADVPRO": "ADV",
    "ANUM": "ADJ",
    "APRO": "DET",
    "COM": "ADJ",
    "CONJ": "SCONJ",
    "INTJ": "INTJ",
    "NONLEX": "X",
    "NUM": "NUM",
    "PART": "PART",
    "PR": "ADP",
    "S": "NOUN",
    "SPRO": "PRON",
    "UNKN": "X",
    "V": "VERB"
}
print("Loading the tokenize_lemmatize_taggize model...")


""""
text = "стекло"
m = Mystem()
analysis = m.analyze(text)
print(analysis)
for item in analysis:
    if 'analysis' in item:
        print(f"Слово: {item['text']}")
        for variant in item['analysis']:
            print(f"  Лемма: {variant['lex']}, Грамм. информация: {variant['gr']}")
    else:
        print(f"Пробел или пунктуация: {item['text']}")
"""
m = Mystem()

Loading the tokenize_lemmatize_taggize model...


In [3]:
#не более 10 слов через пробел. RusVectōrēs сложит вектора положительных слов и вычтет из них 
#отрицательные. Затем он выдаст слова, наиболее близкие к получившемуся вектору. Если вы оставите
# отрицательное поле пустым, RusVectōrēs просто найдет центр лексического кластера, образованного 
# положительными словами.
def tokenize_lemmatize_taggize(input_text):
    output = tag_mystem(text=input_text, mapping=mystem2upos)
    return " ".join(output)

#result = tokenize_lemmatize_taggize("россия священная наша держава") print("test: " , result)

def get_custom_similar_words(model, num_results, positive_str, negative_str, need_print=True):
    positive_words = [word for word in tokenize_lemmatize_taggize(positive_str).split() if word in model]
    negative_words = [word for word in tokenize_lemmatize_taggize(negative_str).split() if word in model]

    ret = np.array([], dtype=str)
    if(need_print):
        print("+", positive_words, "\n-" , negative_words)
    if not positive_words:
         return ret
    
    # Получаются наиболее похожие слова
    for word_099 in model.most_similar(positive=positive_words, negative=negative_words, topn=num_results):
            # word_099 слово + коэффициент косинусной близости 
            processed_string = word_099[0].split("_")[0]
            #if processed_string not in ret:
            ret = np.append(ret, processed_string)
            
    if(need_print):
         print(' '.join(ret))# #print (ret)
         print(" ")

    return ret

In [4]:
_=get_custom_similar_words(model, 99,
                       "бред невнимательность наивность",
                        "трезвость опыт" # <-- это вычитается
                        , need_print=True
                         )

+ ['бред_NOUN', 'невнимательность_NOUN', 'наивность_NOUN'] 
- ['трезвость_NOUN', 'опыт_NOUN']
бестолковость болтливость глупость мнительность простодушие доверчивость забывчивость легкомыслие обидчивость ребячество ребяческий несдержанность сентиментальность несерьезность вздорность небрежность рассеянность преувеличенный непонятность кривляние восторженность непонятливость бестактный невнимание мания бестактность неискренность тупость насмешка карикатурность бесчувственность грубость глупый недогадливость игривость застенчивость боязливость простительный непочтительность чудачество непростительный нелепый нелепость недомыслие кокетство манерность мальчишество вздорный бесхарактерность бравада неумелость поверхностность сумасбродство бессердечность детскость опрометчивость комизм напускной вульгарности непрактичность упрекнуть чепуха наивный вульгарность черствость смешной вздор странность невнимательный нервность жестокость нелепица навязчивость низость галиматья неряшливость бесстыдс

In [5]:
need_print=True

first_list =   get_custom_similar_words(model, 99, "тропинка собака ружье",  ""    , need_print=need_print)
second_list =  get_custom_similar_words(model, 99, "тропинка собака ружье",  "дичь", need_print=need_print)

print("Слова, которые были в первом массиве, но отсутствуют во втором:")
missing_words = set(first_list) - set(second_list)
print(" ".join(missing_words))

+ ['тропинка_NOUN', 'собака_NOUN', 'ружье_NOUN'] 
- []
лошадь тропа куст козуль охотник двустволка гончий путик берданка заяц куропатка пес тропка егерь кабан винтовка дробовик опушка коза легавый засадка олень изюбр чаща палка кабарга волк зверовый ружье дичь овраг борзая медведица загонщик лось лес ягдташа гончая рябчик егерь сохатый гончий ездовый кочка сохатый сторожка гончая речка рогатина зверь валежник козуля просека тетеревь лесок водопой лайка шалаш охотничий карабин пороша борзятник бекас стая глухарь лощина росомаха зимник собачонка собака изгородь нарты косуля лощинка нарт лисица нарта полверст медведь вальдшнеп косач болотец тетеревенка выводка лось стойбище козуля дрофа полянка дорожка чжан-бао тетерево сарай сеттер одностволка дупель утка ружьишко косогор
 
+ ['тропинка_NOUN', 'собака_NOUN', 'ружье_NOUN'] 
- ['дичь_NOUN']
куст тропка тропа палка дорожка лошадь забор винтовка калитка изгородь жердь пес овраг коновязь плетень сарай просека колышек сторожка поводок косогор 